In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[2]").appName("test").getOrCreate()

In [2]:
trace_file = "/Users/vkprabhala/Desktop/twitter-challenge/application-trace.json" 
app_trace = spark.read.json(trace_file,multiLine=True)

In [3]:
app_trace.printSchema()

root
 |-- cesMetadata: struct (nullable = true)
 |    |-- countryCode: string (nullable = true)
 |    |-- oauthAppId: long (nullable = true)
 |-- messageSequenceNumber: long (nullable = true)
 |-- name: string (nullable = true)
 |-- parentSpanId: long (nullable = true)
 |-- spanId: long (nullable = true)
 |-- startTimeMicroseconds: long (nullable = true)
 |-- stopTimeMicroseconds: long (nullable = true)
 |-- traceId: struct (nullable = true)
 |    |-- leastSignificantBits: long (nullable = true)
 |    |-- mostSignificantBits: long (nullable = true)



In [4]:
app_trace.show(4,False)

+-----------+---------------------+-----------------------+------------+-----------+---------------------+--------------------+------------------------------------------+
|cesMetadata|messageSequenceNumber|name                   |parentSpanId|spanId     |startTimeMicroseconds|stopTimeMicroseconds|traceId                                   |
+-----------+---------------------+-----------------------+------------+-----------+---------------------+--------------------+------------------------------------------+
|[BR, 234]  |255372               |time-to-fully-loaded   |null        |-2147472118|1607549809181000     |null                |[8736631216184186000, 7012058579801421000]|
|[BR, 234]  |255397               |time-to-fully-loaded   |null        |-2147472118|null                 |1607549811804000    |[8736631216184186000, 7012058579801421000]|
|[US, 234]  |255256               |loading-direct-messages|-2058295690 |-2147467911|1607549745719000     |null                |[-3946938644639398

In [5]:
app_trace.count()

884000

In [6]:
from pyspark.sql.functions import * 

In [7]:
# How many span messages are in this file?
# How many unique spans do the messages describe?
# How many traces are in this file?
app_trace.agg(count('messageSequenceNumber').alias('num_span_messages'),
              countDistinct('spanId').alias('unique_spans'),
              countDistinct('traceId').alias('num_traces'),
             ).show()

+-----------------+------------+----------+
|num_span_messages|unique_spans|num_traces|
+-----------------+------------+----------+
|           884000|      441987|     34000|
+-----------------+------------+----------+



In [8]:
#What are the distinct names of the spans we are tracking?
app_trace.select('name').distinct().show(100,False)

+----------------------------+
|name                        |
+----------------------------+
|loading-timeline-tweets     |
|authenticating-user         |
|loading-cached-tweets       |
|loading-notifications       |
|time-to-fully-loaded        |
|rendering-cached-tweets     |
|initializing-application    |
|rendering-message-count     |
|loading-direct-messages     |
|rendering-timeline-tweets   |
|finalize-application-start  |
|loading-feature-flags       |
|rendering-notification-count|
+----------------------------+



In [9]:
# What are the names of the traces?
# The root span defines the name of the trace (i.e. the name of the root span is the name of the trace)
app_trace.filter(col('parentSpanId').isNull()).select(col('name')).distinct().show(100,False)

+--------------------+
|name                |
+--------------------+
|time-to-fully-loaded|
+--------------------+



In [10]:
# What is the minimum and maximum duration of all the spans? How about average (mean) and p95?
span_durations = app_trace.groupBy(col('spanId')).agg((max('stopTimeMicroseconds') - max('startTimeMicroseconds')).alias('span_duration'))
min_max_span_durations = span_durations.agg(min('span_duration').alias('min_span_duration'),
                                            max('span_duration').alias('max_span_duration'),
                                            avg('span_duration').alias('mean_span_duration'),
                                            expr('percentile(span_duration, 0.95)').alias('p95_span_duration')
                                            )
min_max_span_durations.show(1,False)                                            

+-----------------+-----------------+------------------+-----------------+
|min_span_duration|max_span_duration|mean_span_duration|p95_span_duration|
+-----------------+-----------------+------------------+-----------------+
|-166000          |3805000          |514873.8922185494 |2365000.0        |
+-----------------+-----------------+------------------+-----------------+



In [11]:
# What is unexpected about these results, and how should we handle it for further analysis? What would you do if you saw something like this in your data?
# Some spans have startTime > stopTime which results in min_span_duration being negative. We can exclude the spans with startTime > stopTime for further analysis. 
# Also there are some spans with more than 1 pair of start and stoptime like the ones shown below 
app_trace.groupBy('spanId').count().filter(col('count') != 2).show(100,False)


+-----------+-----+
|spanId     |count|
+-----------+-----+
|1334505210 |4    |
|289675416  |4    |
|2044164616 |4    |
|-1623396131|4    |
|1388118298 |4    |
|708324914  |4    |
|571003751  |4    |
|1638074620 |4    |
|1851979998 |4    |
|781219535  |4    |
|2056622259 |4    |
|-79912640  |4    |
|495190732  |4    |
+-----------+-----+



In [12]:
# For the rest of the questions, let’s just pretend that we live in a universe where the weird data is acceptable, and include it in further analysis.
# What is the name of the span which has no children, and has the shortest duration on average? Longest duration on average?
span_with_no_child = app_trace.filter(col('parentSpanId').isNull()).select(col('name').alias('span_with_no_children')).distinct()
span_with_no_child.show(10,False)
span_durations = app_trace.groupBy('spanId','name').agg( ((max('stopTimeMicroseconds') - max('startTimeMicroseconds')).alias('span_duration') ))
avg_span_durations = span_durations.groupBy('name').agg(avg('span_duration').alias('avg_span_duration'))
shortest_avg_duration_span = avg_span_durations.orderBy('avg_span_duration').head(1)
longest_avg_duration_span = avg_span_durations.orderBy(desc('avg_span_duration')).head(1)
print(shortest_avg_duration_span)
print(longest_avg_duration_span)

+---------------------+
|span_with_no_children|
+---------------------+
|time-to-fully-loaded |
+---------------------+

[Row(name='finalize-application-start', avg_span_duration=99491.82352941176)]
[Row(name='time-to-fully-loaded', avg_span_duration=2477797.5)]


In [13]:
# you get a report that several users have said the latest version of the app (oauthAppId 234) is slower in comparison to the older version (oauthAppId 123). Other users have said the opposite, that the experience seems faster. Who is right? Why?
# Assumption is we can compare the avg duration of different app versions for each span.
span_app_version_durations = app_trace.groupBy('spanId',(col('cesMetadata')['oauthAppId']).alias('appVersion'),'name') \
                                      .agg(((max('stopTimeMicroseconds') - max('startTimeMicroseconds')).alias('span_duration')))
avg_durations_by_version_name = span_app_version_durations.groupBy('appVersion','name').agg(avg('span_duration').alias('avg_span_duration'))
avg_durations_by_version_name.orderBy('appVersion','avg_span_duration').show(40,False)
#Looking at the results we can say that for each span the avergae duration is slightly less for app version 123 compared to 234. It can be said that the users who reported app 123 being faster than 234 are right based on the average duration metric for different spans. SLightly more difference is noticeable in spans 'initializing-application', 'time-to-fully-loaded'. 

+----------+----------------------------+------------------+
|appVersion|name                        |avg_span_duration |
+----------+----------------------------+------------------+
|123       |finalize-application-start  |99348.78571428571 |
|123       |authenticating-user         |199509.64285714287|
|123       |loading-direct-messages     |298890.28571428574|
|123       |loading-notifications       |298896.71428571426|
|123       |loading-timeline-tweets     |299250.28571428574|
|123       |loading-cached-tweets       |299676.78571428574|
|123       |loading-feature-flags       |299725.5714285714 |
|123       |rendering-timeline-tweets   |397410.35714285716|
|123       |rendering-message-count     |397482.0          |
|123       |rendering-notification-count|397899.78571428574|
|123       |rendering-cached-tweets     |399005.5714285714 |
|123       |initializing-application    |798182.3571428572 |
|123       |time-to-fully-loaded        |2453698.5714285714|
|234       |finalize-app

In [14]:
import pandas as pd 
pandas_df = avg_durations_by_version_name.toPandas()

In [15]:
pivoted = pandas_df.pivot(index='name',columns='appVersion',values='avg_span_duration')
pivoted.plot.bar()